# BBMs for the Power Flow

In this script, we aim to construct Black-Box Models (BBMs) (i.e., data-driven models) for the Power Flow (PF) problem. We use the data stored in the 'data/PF' folder, containing tables with the PF IO data.

In [1]:
import os
from pathlib import Path
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import torch
import sys
from importlib import reload

import greyboxmodels.bbmcpsmodels.creator as creator


# Set the working directory
os.chdir("D:/projects/IPTLC_BBMs/")
print(f"Working directory: {os.getcwd()}")

# Check GPU availability
device = creator.get_device()
print(f"Device: {device}")

Working directory: D:\projects\IPTLC_BBMs
Device: cuda:0


In [5]:
# Specify the paths
datasets_folder = Path("data/PF/20240227_195448")

# # Print the available datasets
# print("Available datasets:")
# for path in datasets_folder.iterdir():
#     if path.is_dir() and "OPF" in path.name:
#         print(path.name)

# Setup the BBM creator

In [6]:
# Create the BBMCreator
reload(creator)
BBM_creator = creator.BBMCreator()

In [7]:
# Set up the dataset
dataset_name = "MinMaxNormalizedPF"
loaders = creator.setup_datasets(datasets_folder,
                                 dataset_name,
                                 remove_nans=True,
                                 ratios=(0.70, 0.15, 0.15),
                                 batch_size=32,
                                 input_name="pf_inputs_minmax_normalized.npy",
                                 output_name="pf_outputs_minmax_normalized.npy")
BBM_creator.set_dataloaders(*loaders)

Loading: data\PF\20240227_195448
---- Dataset loaded ----
Input shape: (50688, 56)
Output shape: (50688, 105)
---- Removing NaNs ----
Rows to delete: []
Input shape: (50688, 56)
Output shape: (50688, 105)
---- Converting to torch tensors ----
---- Dataset loaded! ----


# BBM 1: a two-layer feedforward neural network

In [10]:
from greyboxmodels.bbmcpsmodels.nn.feedforward import BBM1_SimpleNet

In [11]:
# Instantiate model
input_size = loaders[0].dataset[0][0].shape[0]
output_size = loaders[0].dataset[0][1].shape[0]

BBM_creator.instantiate_model(BBM1_SimpleNet, input_size, output_size)

In [12]:
# Train the model
BBM_creator.train(save_to="models", epochs=100)

------ Training model 'BBM1_SimpleNet' ------
Models and summary will be save to 'models/' (will be created if it does not exist)
    - Model path: models\BBM1_SimpleNet_MinMaxNormalizedPF_20240228-154711.pt
    - Summary path: models/models_summary.csv
Training on cuda:0


Epoch 1/100 (Loss - Train: 5.87e-03, Best val: N/A): 100%|██████████| 1109/1109 [00:06<00:00, 176.81it/s]
Epoch 2/100 (Loss - Train: 1.73e-03, Best val: 1.97e-03): 100%|██████████| 1109/1109 [00:04<00:00, 275.64it/s]
Epoch 3/100 (Loss - Train: 1.34e-03, Best val: 1.62e-03): 100%|██████████| 1109/1109 [00:03<00:00, 284.40it/s]
Epoch 4/100 (Loss - Train: 1.11e-03, Best val: 1.34e-03): 100%|██████████| 1109/1109 [00:03<00:00, 293.12it/s]
Epoch 5/100 (Loss - Train: 9.63e-04, Best val: 9.07e-04): 100%|██████████| 1109/1109 [00:03<00:00, 282.45it/s]
Epoch 6/100 (Loss - Train: 8.52e-04, Best val: 9.07e-04): 100%|██████████| 1109/1109 [00:03<00:00, 282.26it/s]
Epoch 7/100 (Loss - Train: 7.61e-04, Best val: 7.46e-04): 100%|██████████| 1109/1109 [00:03<00:00, 288.01it/s]
Epoch 8/100 (Loss - Train: 6.73e-04, Best val: 6.49e-04): 100%|██████████| 1109/1109 [00:04<00:00, 272.39it/s]
Epoch 9/100 (Loss - Train: 5.97e-04, Best val: 5.80e-04): 100%|██████████| 1109/1109 [00:04<00:00, 248.29it/s]
Epoch 

                                                   Train loss  \
Model          Dataset            Timestamp                     
BBM1_SimpleNet MinMaxNormalizedPF 20240228-154711    0.000296   

                                                   Validation loss  Test loss  \
Model          Dataset            Timestamp                                     
BBM1_SimpleNet MinMaxNormalizedPF 20240228-154711         0.000297   0.000303   

                                                   Input size  Output size  \
Model          Dataset            Timestamp                                  
BBM1_SimpleNet MinMaxNormalizedPF 20240228-154711          56          105   

                                                   Training time [ms]  \
Model          Dataset            Timestamp                             
BBM1_SimpleNet MinMaxNormalizedPF 20240228-154711          394038.817   

                                                                                          Model path  
Mo

D:\projects\IPTLC_BBMs\src\greyboxmodels\bbmcpsmodels\creator.py:413: PerformanceWarning: indexing past lexsort depth may impact performance.
  comparison = (model_name, dataset_name, timestamp) in results_table.index


In [13]:
BBM_creator._summary()

,,,Train loss,Validation loss,Test loss,Input size,Output size,Training time [ms],Model path
Model,Dataset,Timestamp,,,,,,,
BBM1_SimpleNet,MinMaxNormalizedPF,20240228-154711,0.000296,0.000299,0.000302,56,105,394038.817,models\BBM1_SimpleNet_MinMaxNormalizedPF_20240...
